In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from datetime import datetime
import os

In [ ]:
DIR = "trials/"

#FILES = ["caffe.wu.vgenty.log.INFO.20160511-184016.16450",
#         "caffe.wu.vgenty.log.INFO.20160511-210724.24028",
#         "caffe.wu.vgenty.log.INFO.20160511-233906.27946",
#         "caffe.wu.vgenty.log.INFO.20160512-080304.9249",
#         "caffe.wu.vgenty.log.INFO.20160512-165623.22519",
#         "caffe.wu.vgenty.log.INFO.20160512-191117.2182"]

#FILES=["caffe.wu.vgenty.log.INFO.20160515-164420.7115",
#       "caffe.wu.vgenty.log.INFO.20160516-000242.9477",
#       "caffe.wu.vgenty.log.INFO.20160516-094927.31426"]
FILES = ["caffe.wu.vgenty.log.INFO.20160517-164907.22335",
         "caffe.wu.vgenty.log.INFO.20160517-170439.3152"]
FILES = [os.path.join(DIR,f) for f  in FILES]
#for F in FILES:
#    os.system("scp vgenty@wu.nevis.columbia.edu:/tmp/{} .".format(F))

In [ ]:
read = []
for F in FILES:
    with open(F) as f:
        read.append(f.read())

iterations = [r.split("solver.cpp:228]")[1:] for r in read]

In [ ]:
frames = []
for ix,iteration in enumerate(iterations):
    fig,ax=plt.subplots(figsize=(10,6))

    rr = {"Iteration" : re.compile("Iteration\s(\d+)\,"),
          "loss"      : re.compile("\,\sloss\s\=\s(\d+\.\d+(e\-\d+)?)\s"),
          "acc"       : re.compile("acc\s\=\s(\d+\.*\d*(e\-\d+)?)\s")}
    di = {"Iteration" : [],
          "loss" : [],
          "acc" : []}

    for i in iteration:
        for r in rr:
            a = rr[r].search(i)
            if a is None: 
                continue
            a = a.group(1)   
            a = float(a)
            di[r].append(a)

    # converting to series means pandas will be O.K. with unequal length
    # lists
    for key,val in di.iteritems():
        di[key] = pd.Series(val)

    df = pd.DataFrame.from_dict(di)   
    #############################
    frames.append(df)
    print df.shape
    matplotlib.rcParams['font.size'] = 16
    
    batch = 64
    ntotal = 2*105786

    ax.plot(df.Iteration.values*batch/ntotal,
            df.loss.rolling(window=1).mean(),'-',color='blue',lw=2,label='loss',alpha=0.6)
    ax.plot(df.Iteration.values*batch/ntotal,
            df.loss.rolling(window=10).mean(),'-',color='red',lw=2,label='avg')

    ax2 = ax.twinx()
    ax2.plot(df.Iteration.values*batch/ntotal,
             df.acc.rolling(window=1).mean(),'-',label='acc',color='green',lw=2,alpha=0.6)
    ax2.plot(df.Iteration.values*batch/ntotal,
             df.acc.rolling(window=10).mean(),'-',label='avg',color='orange',lw=2)

    #ax.legend(loc='best')

    ax.set_xlabel("Epoch",fontweight='bold')
    ax.set_ylabel("Loss",fontweight='bold')
    ax.set_ylim(0.0,10.0)
    ax2.set_ylim(0.0,1.0)
    ax2.set_ylabel("Accuracy",fontweight='bold')
    ax.legend(loc='upper left')
    ax2.legend(loc='upper right')
    plt.savefig('b_{}.png'.format(ix), format='png', dpi=500)
    plt.show()

In [ ]:
scores = sorted([S for S in os.listdir(".") if S.endswith(".txt") and S.startswith("13") or S.startswith("14")],key=lambda s : int(s[:2]))
print scores
DF = [pd.read_csv(S,sep=' ',header=None,names=['file']+list(np.arange(1,6,1))) for S in scores]

import re
N = re.compile("iter\_([0-9]+).caffemodel")
def iteration(row):
    return N.search(row['file']).group(1)

for D in DF:
    D['iteration'] = D.apply(iteration,axis=1)
    D.drop('file',1,inplace=True)
    D.drop(5,1,inplace=True)

DFF = pd.concat(DF)

In [ ]:
prevRow = 0
cc = 0
series = {}
series['iter'] = []
series['acc'] = []

for ix,row in enumerate(DFF.iterrows()):    
    ii = float(row[1][-1])
    mean = float(row[1][0:-1].mean())

    if prevRow < ii:
        diff = ii - prevRow
        prevRow = ii
    else:
        diff = ii
        prevRow = ii

    cc+=diff
    series["iter"].append(cc)
    series["acc"].append(mean)

val_df = pd.DataFrame(series)
print val_df['iter'].values

In [ ]:
cdf = pd.concat(frames)

In [ ]:
matplotlib.rcParams['font.family'] = 'serif'
fig,ax=plt.subplots(figsize=(10,6))
ax.plot(np.arange(0,cdf.index.values.size,1.0)*batch/ntotal,
         cdf.acc.rolling(window=1).mean(),color='green',alpha=0.5,label='train acc per batch (64)')
ax.plot(np.arange(0,cdf.index.values.size,1.0)*batch/ntotal,
         cdf.acc.rolling(window=20).mean(),color='orange',alpha=1,label='avg train acc (20 iters)')
ax.plot(val_df['iter'].values*batch/ntotal,
        val_df['acc'].values,'-',label='test acc (100 random)',alpha=0.9)
#ax.axvline(x=0.6,color="purple")
#ax.axvline(x=1.2,color="purple")
#ax.axvline(x=1.2+1.7,color="purple")
#ax.axvline(x=1.2+1.7+1.7,color="purple")
#ax.axvline(x=1.2+1.7+1.7+2.0,color="purple")
#ax.axvline(x=1.2+1.7+1.7+2.0+0.5,color="purple")
ax.set_xlabel("Epoch",fontweight='bold')
ax.set_ylabel("Acc",fontweight='bold')
ax.set_ylim(0.0,1.0)
ax.legend(loc='lower right')
ax.grid()
plt.savefig('a_acc7.png'.format(ix), format='png', dpi=500)
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(10,6))
ax.plot(np.arange(0,cdf.index.values.size,1.0)*batch/ntotal,
         cdf.loss.rolling(window=1).mean(),color='blue',alpha=0.5)
ax.plot(np.arange(0,cdf.index.values.size,1.0)*batch/ntotal,
         cdf.loss.rolling(window=20).mean(),color='red')
#ax.axvline(x=0.6,color="purple")
#ax.axvline(x=1.2,color="purple")
#ax.axvline(x=1.2+1.7,color="purple")
#ax.axvline(x=1.2+1.7+1.7,color="purple")
#x.axvline(x=1.2+1.7+1.7+2.0,color="purple")
#ax.axvline(x=1.2+1.7+1.7+2.0+0.5,color="purple")
ax.set_xlabel("Epoch",fontweight='bold')
ax.set_ylabel("Loss",fontweight='bold')

ax.set_ylim(0.0,3.0)
ax.grid()
plt.savefig('a_loss7.png'.format(ix), format='png', dpi=500)
plt.show()

In [ ]:
val_df.to_csv("google_lowres.txt")